In [143]:
import os
import sys
from collections import Counter
from typing import Optional, AbstractSet, Generator
from time import sleep
import subprocess
import logging
import pandas as pd
import numpy as np
import multiprocessing as mp

In [2]:
ONTOLOGY_PATH = "/Volumes/LittleJim/Datasets/AudioSet/ontology.json"
DATASET_PATH = "/Volumes/LittleJim/Datasets/AudioSet/unbalanced_train_segments.csv"
DOWNLOAD_DIR = "/Volumes/LittleJim/Datasets/youtube/"

In [3]:
ontology = pd.read_json(ONTOLOGY_PATH)
ontology = ontology.set_index('id')

In [5]:
genres = ontology[ontology.name.str.contains("genre")].child_ids.iloc[0]
moods = ontology[ontology.name.str.contains("mood")].child_ids.iloc[0]
instruments = ontology[ontology.name.str.contains("Musical instrument")].child_ids.iloc[0]
roles = ontology[ontology.name.str.contains("Music role")].child_ids.iloc[0]

In [6]:
genre_id = ontology[ontology.name.str.contains("genre")].index[0]
mood_id = ontology[ontology.name.str.contains("mood")].index[0]
instrument_id = ontology[ontology.name.str.contains("Musical instrument")].index[0]
role_id = ontology[ontology.name.str.contains("Music role")].index[0]

In [7]:
def ontology_leaves(root_id):
    children = []
    def dfs(current):
        child_ids = ontology.loc[current, "child_ids"]
        if not child_ids:
            children.append(current)
        else:
            for child in child_ids:
                dfs(child)
    dfs(root_id)
    return children

In [19]:
def ontology_all_children(root_id):
    children = set()
    def dfs(current):
        child_ids = ontology.loc[current, "child_ids"]
        children.add(current)
        if not child_ids:
            return
        else:
            for child in child_ids:
                dfs(child)
    dfs(root_id)
    return children

In [72]:
all_mood_ids = ontology_all_children(id_from_label("Music mood"))

In [37]:
ontology[ontology.name == "Rock music"].index.values[0]

'/m/06by7'

In [53]:
id_from_label("Music")

'/m/04rlf'

In [55]:
dataset

,# YTID,start_seconds,end_seconds,positive_labels
0,---1_cCGK4M,0.0,10.0,"{/m/07jdr, /m/06d_3, /m/0284vy3, /m/07rwm0c, /..."
1,---2_BBVHAA,30.0,40.0,{/m/09x0r}
2,---B_v8ZoBY,30.0,40.0,{/m/04rlf}
3,---EDNidJUA,30.0,40.0,"{/m/02qldy, /m/05zppz, /m/02zsn, /m/09x0r}"
4,---N4cFAE1A,21.0,31.0,"{/m/04rlf, /m/09x0r}"
...,...,...,...,...
2041784,zzyyleHsxfk,20.0,30.0,"{/m/0bt9lr, /m/05tny_, /m/068hy, /m/0jbk}"
2041785,zzz-JsGPtxQ,120.0,130.0,"{/m/0l14jd, /m/015lz1}"
2041786,zzz3PZXRQ_8,30.0,40.0,"{/m/030rvx, /m/09x0r}"
2041787,zzznDcamMpw,0.0,10.0,"{/m/09ddx, /m/09x0r}"


In [56]:
all_music_ids = ontology_all_children(id_from_label("Music"))

In [59]:
music = dataset[~dataset.positive_labels.map(lambda lab: lab.isdisjoint(all_music_ids))]

In [60]:
music

,# YTID,start_seconds,end_seconds,positive_labels
2,---B_v8ZoBY,30.0,40.0,{/m/04rlf}
4,---N4cFAE1A,21.0,31.0,"{/m/04rlf, /m/09x0r}"
6,---g9OGAhwc,30.0,40.0,"{/m/0c1dj, /m/04rlf}"
8,---mO--kRQk,30.0,40.0,{/m/04rlf}
12,--0AzKXCHj8,22.0,32.0,{/m/04rlf}
...,...,...,...,...
2041778,zzy_5evgl3Y,30.0,40.0,"{/m/04rlf, /m/09x0r}"
2041779,zzycbwlBJXQ,180.0,190.0,"{/m/04rlf, /m/0164x2}"
2041782,zzyr6l8MetA,60.0,70.0,"{/m/05148p4, /m/04rlf}"
2041785,zzz-JsGPtxQ,120.0,130.0,"{/m/0l14jd, /m/015lz1}"


In [67]:
def size_of_class(label):
    id_set = {id_from_label(label)}
    samples = music[~music.positive_labels.map(lambda lab: lab.isdisjoint(id_set))]
    return len(samples)

In [75]:
for label in ontology[ontology.index.isin(all_roles)].name.values:
    print(size_of_class(label), label)

4835 Background music
4220 Theme music
1318 Jingle (music)
6510 Soundtrack music
3127 Lullaby
3249 Video game music
2017 Christmas music
4782 Dance music
691 Wedding music
0 Birthday music


In [74]:
for label in ontology[ontology.index.isin(all_instruments)].name.values:
    print(size_of_class(label), label)

6549 Choir
179 Cowbell
11834 Electric guitar
6321 Bass guitar
14416 Acoustic guitar
2700 Steel guitar, slide guitar
1686 Tapping (guitar technique)
13879 Strum
2336 Banjo
1461 Sitar
2252 Mandolin
1095 Zither
5172 Ukulele
0 Clavinet
0 Rhodes piano
1201 Electronic organ
1277 Hammond organ
5336 Sampler
0 Mellotron
1957 Harpsichord
3219 Drum machine
4365 Rimshot
2838 Drum roll
9116 Bass drum
1246 Timpani
1603 Tabla
3780 Hi-hat
0 Crash cymbal
2039 Wood block
625 Tambourine
474 Maraca
364 Gong
393 Tubular bells
5361 Marimba, xylophone
3989 Glockenspiel
1402 Vibraphone
869 Steelpan
8294 Orchestra
1099 French horn
3707 Trumpet
2666 Trombone
0 Cornet
0 Bugle
1072 String section
3424 Pizzicato
5148 Cello
2153 Double bass
4659 Flute
0 Alto saxophone
0 Soprano saxophone
1998 Clarinet
0 Oboe
0 Bassoon
1921 Harp
1074 Church bell
1062 Jingle bell
104 Bicycle bell
80 Tuning fork
447 Wind chime
545 Change ringing (campanology)
2095 Harmonica
2772 Accordion
1655 Bagpipes
1120 Didgeridoo
209 Shofar
511 T

In [73]:
for label in ontology[ontology.index.isin(all_mood_ids)].name.values:
    print(size_of_class(label), label)

0 Music mood
1338 Happy music
1035 Funny music
1650 Sad music
3971 Tender music
5518 Exciting music
985 Angry music
1617 Scary music


In [69]:
for label in ontology[ontology.index.isin(all_children_genre)].name.values:
    print(size_of_class(label), label)

1594 Chant
4783 Mantra
0 Music genre
8661 Pop music
7618 Hip hop music
0 Grime music
0 Trap music
2362 Beatboxing
8314 Rock music
6330 Heavy metal
3949 Punk rock
1896 Grunge
3399 Progressive rock
9175 Rock and roll
2177 Psychedelic rock
5007 Rhythm and blues
2799 Soul music
3107 Reggae
0 Dub
5409 Country
1518 Swing music
2600 Bluegrass
3944 Funk
2099 Folk music
1968 Middle Eastern music
4810 Jazz
4011 Disco
6753 Classical music
2157 Opera
38789 Electronic music
6487 House music
17109 Techno
18541 Dubstep
0 Electro
3616 Drum and bass
0 Oldschool jungle
5294 Electronica
4299 Electronic dance music
2715 Ambient music
0 Drone music
4353 Trance music
0 Noise music
0 UK garage
4021 Music of Latin America
0 Cumbia
2437 Salsa music
0 Soca music
0 Kuduro
0 Funk carioca
2244 Flamenco
4643 Blues
3389 Music for children
3042 New-age music
3539 Vocal music
1796 A capella
2752 Music of Africa
2579 Afrobeat
0 Kwaito
2992 Christian music
4164 Gospel music
3795 Music of Asia
1539 Carnatic music
2405 Mu

In [68]:
all_children_genre = ontology_all_children(genre_id)
ontology[ontology.index.isin(all_children_genre)].name.values

array(['Chant', 'Mantra', 'Music genre', 'Pop music', 'Hip hop music',
       'Grime music', 'Trap music', 'Beatboxing', 'Rock music',
       'Heavy metal', 'Punk rock', 'Grunge', 'Progressive rock',
       'Rock and roll', 'Psychedelic rock', 'Rhythm and blues',
       'Soul music', 'Reggae', 'Dub', 'Country', 'Swing music',
       'Bluegrass', 'Funk', 'Folk music', 'Middle Eastern music', 'Jazz',
       'Disco', 'Classical music', 'Opera', 'Electronic music',
       'House music', 'Techno', 'Dubstep', 'Electro', 'Drum and bass',
       'Oldschool jungle', 'Electronica', 'Electronic dance music',
       'Ambient music', 'Drone music', 'Trance music', 'Noise music',
       'UK garage', 'Music of Latin America', 'Cumbia', 'Salsa music',
       'Soca music', 'Kuduro', 'Funk carioca', 'Flamenco', 'Blues',
       'Music for children', 'New-age music', 'Vocal music', 'A capella',
       'Music of Africa', 'Afrobeat', 'Kwaito', 'Christian music',
       'Gospel music', 'Music of Asia', 'Carn

In [8]:
all_genres = ontology_leaves(genre_id)
all_moods = ontology_leaves(mood_id)
all_instruments = ontology_leaves(instrument_id)
all_roles = ontology_leaves(role_id)

In [9]:
genres_df = ontology[ontology.index.isin(all_genres)]
moods_df = ontology[ontology.index.isin(all_moods)]
instruments_df = ontology[ontology.index.isin(all_instruments)]
roles_df = ontology[ontology.index.isin(all_roles)]

In [10]:
for category in [genres_df, moods_df, instruments_df, roles_df]:
    print(category.name.values)

['Mantra' 'Pop music' 'Grime music' 'Trap music' 'Beatboxing'
 'Heavy metal' 'Punk rock' 'Grunge' 'Progressive rock' 'Rock and roll'
 'Psychedelic rock' 'Rhythm and blues' 'Soul music' 'Dub' 'Swing music'
 'Bluegrass' 'Funk' 'Folk music' 'Middle Eastern music' 'Jazz' 'Disco'
 'Opera' 'House music' 'Techno' 'Dubstep' 'Electro' 'Oldschool jungle'
 'Electronica' 'Electronic dance music' 'Drone music' 'Trance music'
 'Noise music' 'UK garage' 'Cumbia' 'Salsa music' 'Soca music' 'Kuduro'
 'Funk carioca' 'Flamenco' 'Blues' 'Music for children' 'New-age music'
 'A capella' 'Afrobeat' 'Kwaito' 'Gospel music' 'Carnatic music'
 'Music of Bollywood' 'Ska' 'Traditional music' 'Independent music']
['Happy music' 'Funny music' 'Sad music' 'Tender music' 'Exciting music'
 'Angry music' 'Scary music']
['Choir' 'Cowbell' 'Electric guitar' 'Bass guitar' 'Acoustic guitar'
 'Steel guitar, slide guitar' 'Tapping (guitar technique)' 'Strum' 'Banjo'
 'Sitar' 'Mandolin' 'Zither' 'Ukulele' 'Clavinet' 'Rhodes p

In [42]:
def split_and_set(string):
    return set(string.split(","))

def exactly_one_match(query, keys):
    """ Returns a matching key value iff exactly one of the query set is in the keys, otherwise pd.NA """
    matches = query.intersection(keys)
    return matches.pop() if len(matches) > 1 else pd.NA

In [43]:
dataset = pd.read_csv(DATASET_PATH, header=2, sep=",", skipinitialspace=True, quotechar='"')

In [44]:
dataset.positive_labels = dataset.positive_labels.map(split_and_set)

In [45]:
genres_set, moods_set, instruments_set, roles_set = map(lambda df: set(df.index), (genres_df, moods_df, instruments_df, roles_df))

In [46]:
def conditional_dataset(query_set):
    return dataset[~dataset.positive_labels.map(lambda lab: exactly_one_match(query_set, lab)).isna()]

In [47]:
genres_dataset, moods_dataset, instruments_dataset, roles_dataset = map(conditional_dataset, (genres_set, moods_set, instruments_set, roles_set))

In [49]:
ontology

,name,description,citation_uri,positive_examples,child_ids,restrictions
id,,,,,,
/m/0dgw9r,Human sounds,Sounds produced by the human body through the ...,,[],"[/m/09l8g, /m/01w250, /m/09hlz4, /m/0bpl036, /...",[abstract]
/m/09l8g,Human voice,The human voice consists of sound made by a hu...,http://en.wikipedia.org/wiki/Human_voice,[],"[/m/09x0r, /m/07p6fty, /m/03qc9zr, /m/02rtxlg,...",[abstract]
/m/09x0r,Speech,Speech is the vocalized form of human communic...,http://en.wikipedia.org/wiki/Speech,"[youtu.be/8uI9H5jGRV8?start=30&end=40, youtu.b...","[/m/05zppz, /m/02zsn, /m/0ytgt, /m/01h8n0, /m/...",[]
/m/05zppz,"Male speech, man speaking",Speech uttered by an adult male human.,,"[youtu.be/6niRPYpLOpQ?start=30&end=40, youtu.b...",[],[]
/m/02zsn,"Female speech, woman speaking",Speech uttered by an adult female human.,,"[youtu.be/4l05nCOnIRg?start=30&end=40, youtu.b...",[],[]
...,...,...,...,...,...,...
/m/025l19,Recording,A sound that appears to come from a recording ...,http://en.wikipedia.org/wiki/Recording,[],[/m/07hvw1],[blacklist]
/m/07hvw1,Field recording,A sound that appears to have been recorded in ...,http://en.wikipedia.org/wiki/Field_recording,"[youtu.be/Gxc0gxJpato?start=490&end=500, youtu...",[],[blacklist]
/m/0174nj,Gramophone record,A sound which appears to come from a gramophon...,http://en.wikipedia.org/wiki/Gramophone_record,[],[],[blacklist]


In [50]:
def id_from_label(label):
    return ontology[ontology.name == label].index.values[0]

In [48]:
genres_dataset

,# YTID,start_seconds,end_seconds,positive_labels
548,--WxoinaQzk,30.0,40.0,"{/m/04rlf, /m/064t9, /m/02x8m}"
748,--jx9vb7eKE,30.0,40.0,"{/m/07s72n, /m/04rlf, /m/07lnk, /m/03mb9, /m/0..."
762,--kYBfYfwxw,30.0,40.0,"{/m/07gxw, /m/04rlf, /m/0gywn, /m/09x0r}"
953,--x2hrwBV1s,30.0,40.0,"{/m/04rlf, /m/03_d0, /m/02x8m, /m/0ggx5q}"
1184,-0CQt5_1Zag,30.0,40.0,"{/m/04rlf, /m/0gywn, /m/02x8m}"
...,...,...,...,...
2040584,zwooPj5G_OQ,30.0,40.0,"{/m/04rlf, /m/064t9, /t/dd00035, /m/06j6l}"
2040679,zx2zVGO9Ueo,30.0,40.0,"{/m/04rlf, /m/07sbbz2, /m/05r6t, /m/06j6l, /m/..."
2040834,zxSlO0YgqLc,230.0,240.0,"{/m/04rlf, /m/06rqw, /m/04szw, /m/015y_n}"
2041213,zyVWKtk_RpE,40.0,50.0,"{/m/02lkt, /m/04rlf, /m/03mb9, /m/026z9}"


In [17]:
genres_dataset

,# YTID,start_seconds,end_seconds,positive_labels
103,--5DNFWcdC4,2.0,12.0,"{/m/07s72n, /m/04rlf, /m/09x0r}"
117,--6HJzEIsFs,30.0,40.0,"{/m/04rlf, /m/06j6l}"
130,--7MeTMkd4s,30.0,40.0,"{/m/04rlf, /m/05rwpb}"
161,--9hKb7IkVY,60.0,70.0,"{/m/04rlf, /m/03lty}"
166,--A4Xbd8gCw,120.0,130.0,"{/m/04rlf, /m/0fx80y, /m/0342h, /m/0ln16, /m/0..."
...,...,...,...,...
2041729,zzrVVh-fltw,50.0,60.0,{/m/0dls3}
2041734,zzt2Oc3Niuk,0.0,10.0,"{/m/07s72n, /m/02lkt, /m/04rlf}"
2041751,zzvNX7W5VIo,180.0,190.0,"{/m/04rlf, /m/0gywn}"
2041755,zzvZ7CxKP7Y,60.0,70.0,"{/m/04rlf, /m/0l14gg, /m/02p0sh1}"


In [17]:
instruments_dataset = instruments_dataset.sample(frac=1)

In [27]:
import os
from typing import Optional, Generator
import pandas as pd
import numpy as np

ONTOLOGY_PATH = "/Volumes/LittleJim/Datasets/AudioSet/ontology.json"
DATASET_PATH = "/Volumes/LittleJim/Datasets/AudioSet/unbalanced_train_segments.csv"
TENSOR_DIR = '/Volumes/LittleJim/Datasets/youtube-tensors/'

def make_ontology_df(ontology_path: str) -> pd.DataFrame:
    ontology = pd.read_json(ontology_path)
    return ontology.set_index('id')

def make_dataset_df(dataset_path):
    return pd.read_csv(dataset_path, header=2, sep=",", skipinitialspace=True, quotechar='"')

def yid_from_path(path: str) -> str:
    _, yid = os.path.split(path)
    tail = "ext"
    while tail:
        yid, tail = os.path.splitext(yid)
    return yid

def path_from_yid(root: str, yid: str, suffix: str) -> Optional[str]:
    path = os.path.join(root, yid, suffix)
    return path if os.path.isfile(path) else None

def paths_in_dir(dirname: str) -> Generator[str, None, None]:
    for root, _, files in os.walk(dirname):
        for name in files:
            yield os.path.join(root, name)

def yids_in_dir(dirname: str) -> Generator[str, None, None]:
    for path in paths_in_dir(dirname):
        if path[-3:] == ".pt":
            yield yid_from_path(path), os.path.split(path)[1]

def oid_from_label(label):
    return ontology[ontology.name == label].index.values[0]

def label_from_oid(ont_id: str) -> str:
    return ontology.loc[ont_id, 'name']

def ontology_leaves(root_id):
    children = []
    def dfs(current):
        child_ids = ontology.loc[current, "child_ids"]
        if not child_ids:
            children.append(current)
        else:
            for child in child_ids:
                dfs(child)
    dfs(root_id)
    return children
            
def make_tensor_df(dirname: str, category: str, category_id: int, source_df: pd.DataFrame) -> pd.DataFrame:
    yids = {yid: path for yid, path in yids_in_dir(dirname)}
    common_df = source_df[source_df["# YTID"].isin(yids)]
    labels = ontology_leaves(oid_from_label(category))
    label_ints = {labels[n]: n for n in range(len(labels))}
    dfs = []
    for label in labels:
        matched_df = common_df[common_df.positive_labels.map(lambda pos: label in pos)][["# YTID"]]
        matched_df = matched_df.rename(columns={"# YTID": "ytid"})
        matched_df["path"] = matched_df["ytid"].map(yids)
        matched_df["label_n"] = label_ints[label]
        matched_df["category_n"] = category_id
        matched_df["label_ontology"] = label
        matched_df["label"] = label_from_oid(label)
        matched_df["category"] = category
        dfs.append(matched_df.reset_index(drop=True))
    ret = pd.concat(dfs).reset_index(drop=True)
    return ret

def save_tensor_df(path: str, df: pd.DataFrame) -> None:
    df.to_csv(path, index=False)

In [28]:
TENSOR_DIR = '/Volumes/LittleJim/Datasets/youtube-tensors/'
dataset = make_dataset_df(DATASET_PATH)
ontology = make_ontology_df(ONTOLOGY_PATH)
categories = {
    "Music genre": 0,
    "Music mood": 1,
    "Musical instrument": 2,
    "Music role": 3
}
tensor_dfs = []

for category in categories:
    df = make_tensor_df(TENSOR_DIR, category, categories[category], dataset)
    name = category.split(" ")[-1]
    save_tensor_df(f"{name}.csv", df)


In [26]:
"Music genre".split(" ")[-1]

'genre'

In [103]:
tensor_yids = list(ids_in_dir(''))

7994

In [ ]:
paths = os.walk(

In [18]:
def download(url, filename, start, end):
    command = [
        "youtube-dl",
        "--no-overwrites",
        "-x",
        "--postprocessor-args",
        f'"-ss {start} -to {end}"',
        "-o",
        filename,
        f'"{url}"',
    ]
    to_run = " ".join(command)
    backoff = 0.5
    while True:
        stdout = !$to_run
        if any(["429:" in line for line in stdout]):
            print(stdout)
            sleep(backoff)
            backoff *= 2
        else:
            break
    return stdout

In [19]:
def write_batch_for_youtubedl(dataset_df):
    urls = list("https://www.youtube.com/watch?v=" + dataset_df["# YTID"])
    filenames = list(DOWNLOAD_DIR + dataset_df["# YTID"] + ".mp3")
    start = list(dataset_df["start_seconds"])
    end = list(dataset_df["end_seconds"])
    return zip(urls, filenames, start, end)

In [20]:
count = Counter(['done'])

In [21]:
def download_if_not_exists(url, filename, start, end):
    if count['done'] % 200 == 0:
        print(count['done'])
    if any(
        (os.path.exists(filename),
        os.path.exists(filename[:-3] + "opus"),
        os.path.exists(filename[:-3] + "m4a"),)
    ):
        count['done'] += 1
        return
    else:
        download(url, filename, start, end)
        count['done'] += 1
        return

In [149]:
write_batch_for_youtubedl(moods_dataset, 'moods', '.')

In [32]:
ontology[ontology.id.isin(moods)]

,id,name,description,citation_uri,positive_examples,child_ids,restrictions
334,/t/dd00031,Happy music,Music that evokes or conveys feelings of happi...,,[],[],[]
335,/t/dd00032,Funny music,Music that evokes or conveys amusement.,,[],[],[]
336,/t/dd00033,Sad music,Music that evokes or conveys feelings of sadness.,,[],[],[]
337,/t/dd00034,Tender music,Music that evokes or conveys feelings of tende...,,[],[],[]
338,/t/dd00035,Exciting music,Music that evokes or conveys feelings of excit...,,[],[],[]
339,/t/dd00036,Angry music,Music that evokes or conveys feelings of anger.,,[],[],[]
340,/t/dd00037,Scary music,Music that evokes or conveys feelings of fear.,,[],[],[]


In [39]:
ontology[ontology.id.isin(ontology[ontology.id.isin(instruments)].iloc[2, -2])]

,id,name,description,citation_uri,positive_examples,child_ids,restrictions
172,/m/05r5c,Piano,Sounds of a musical instrument played via a ro...,http://en.wikipedia.org/wiki/Piano,"[youtu.be/jETQAE_vBQI?start=30&end=40, youtu.b...",[/m/01s0ps],[]
176,/m/013y1f,Organ,Sounds of a keyboard instrument of one or more...,http://en.wikipedia.org/wiki/Organ_(music),"[youtu.be/ddoz52PLnnM?start=480&end=490, youtu...","[/m/03xq_f, /m/03gvt]",[]
179,/m/0l14qv,Synthesizer,Sounds associated with an electronic musical i...,http://en.wikipedia.org/wiki/Synthesizer,"[youtu.be/DQ5KwZ2qBMY?start=390&end=400, youtu...","[/m/01v1d8, /m/0gkd1]",[]
182,/m/03q5t,Harpsichord,Sounds of a keyboard-controlled instrument in ...,http://en.wikipedia.org/wiki/Harpsichord,"[youtu.be/3QyBsEwyI4g?start=230&end=240, youtu...",[],[]


In [34]:
ontology[ontology.id.isin(roles)]

,id,name,description,citation_uri,positive_examples,child_ids,restrictions
323,/m/025td0t,Background music,Styles of music or soundscapes primarily inten...,http://en.wikipedia.org/wiki/Background_music,[],[],[]
324,/m/02cjck,Theme music,Music often written specifically for a radio p...,http://en.wikipedia.org/wiki/Theme_music,[],[],[]
325,/m/03r5q_,Jingle (music),A short song or tune used in advertising and f...,http://en.wikipedia.org/wiki/Jingle,[],[],[]
326,/m/0l14gg,Soundtrack music,"Recorded music accompanying a movie, video, TV...",http://en.wikipedia.org/wiki/Soundtrack,[],[],[]
327,/m/07pkxdp,Lullaby,The act of singing a quiet song to lull a chil...,http://wordnetweb.princeton.edu/perl/webwn?s=l...,[],[],[]
328,/m/01z7dr,Video game music,The soundtrack that accompanies video games. E...,http://en.wikipedia.org/wiki/Video_game_music,[],[],[]
329,/m/0140xf,Christmas music,A variety of genres of music normally performe...,http://en.wikipedia.org/wiki/Christmas_music,[],[],[]
330,/m/0ggx5q,Dance music,Music composed specifically to facilitate or a...,http://en.wikipedia.org/wiki/Dance_music,[],[],[]
331,/m/04wptg,Wedding music,"Music played at wedding celebrations, includin...",http://en.wikipedia.org/wiki/Wedding_music,[],[],[]
332,/t/dd00029,Birthday music,Music specifically used or performed at birthd...,,[],[],[]
